In [ ]:
import os

base_path = "C:\\Projects\\ature\\" # WINDOWS
# base_path = "home/ak/Projects/ature/" # LINUX

data_file_path = base_path + "\\data\\av_wide_data_set"
out_path = base_path + "\\out\\wide"
    
os.chdir(base_path)

from commons.IMAGE import Image
from commons.ImgLATTICE import Lattice
import preprocess.utils.img_utils as imgutils
from commons.MAT import Mat
from PIL import Image as IMG
import numpy as np
from commons import constants as const
import cv2
from preprocess.algorithms import fast_mst as fmst

In [ ]:
def run_segmentation(img_obj, lattice_obj, params, test_image):
    
    ##### Unpack all params
    SKELETONIZE_THRESHOLD, IMG_LATTICE_COST_ASSIGNMENT_ALPHA, IMG_LATTICE_COST_GABOR_IMAGE_CONTRIBUTION, SEGMENTATION_THRESHOLD = params
    
    ##### Create skeleton based on threshold
    img_obj.create_skeleton(threshold=SKELETONIZE_THRESHOLD, kernels=imgutils.get_chosen_skeleton_filter())
    seed_node_list = imgutils.get_seed_node_list(img_obj.img_skeleton)

    ##### Run segmnetation
    graph = fmst.run_segmentation(image_object=img_obj,
                          lattice_object=lattice_obj,
                          seed_list=seed_node_list,
                          segmentation_threshold=SEGMENTATION_THRESHOLD,
                          alpha=IMG_LATTICE_COST_ASSIGNMENT_ALPHA,
                          img_gabor_contribution=IMG_LATTICE_COST_GABOR_IMAGE_CONTRIBUTION,
                          img_original_contribution=1-IMG_LATTICE_COST_GABOR_IMAGE_CONTRIBUTION)

In [ ]:
############# ENTRY POINT HERE ###############
############################################
params = (70,   ### SKELETONIZE_THRESHOLD
          5.0,    ### IMG_LATTICE_COST_ASSIGNMENT_ALPHA
          0.7,  ### IMG_LATTICE_COST_GABOR_IMAGE_CONTRIBUTION
          7.0)  ### SEGMENTATION_THRESHOLD

#### Work on all images in a directory
os.chdir(data_file_path)
for test_image in os.listdir(os.getcwd()):
    
    print('### WORKING ON: ' + test_image)
    file = Mat(file_name=test_image)
    original = file.get_image('I2')
    img_obj = Image(image_arr=original[:,:,1])
    
    img_obj.apply_bilateral()
    img_obj.apply_gabor(kernel_bank=imgutils.get_chosen_gabor_bank()) 
    print('Filter applied.')
    
    lattice_obj = Lattice(image_arr_2d=img_obj.img_gabor)
    lattice_obj.generate_lattice_graph()
    print('Lattice created.')

    run_segmentation(img_obj, lattice_obj, params, test_image)
    
    os.chdir(out_path)
    IMG.fromarray(lattice_obj.accumulator).save(test_image + '.JPEG')
    IMG.fromarray(original).save(test_image + '-ORIGINAL.JPEG')
    
    print('Done')